# Introduction

The ZEN Temple is a webserver with the purpose to provide a user-friendly API for working with results from ZEN Garden.

You need to specify some environment variables by renaiming the `.env.example` file and overwrite the variables specified.
Currently, only the variable `SOLUTION_FOLDER` is relevant which specifies the folder that contains the results from the ZEN Garden module. 

By default, this is `./public/outputs` and you can simply create this folder and copy the solutions that you want to explore to it. Otherwise you can also change the variable to a custom folder. Be aware though that the server will create new Numpy-Files that store the values of the different components!

To start the server, you can use the following command `uvicorn src.main:app --reload --host 0.0.0.0` which starts the server in development mode. Be sure that you installed the dependencies first with `pip install -r requirements.txt`, preferably in a virtual environment.


# Usage

Once the server is running, you can try out if everything is working by heading over to `http://localhost:8000/docs` in your browser. It should show the Swagger-Documentation of all endpoints. Alternatively we can test it with Pythons `requests` module:

In [ ]:
import requests

base_url = "http://localhost:8000"

requests.get(f"{base_url}/docs")

If this works, the local version of your server is running. However, we will change to the public server for now to be sure that the solutions in this example exist:

In [ ]:
base_url = "https://zen-land.ethz.ch"

The HTTP-response code 200 confirms that everything is working. As mentioned above, when you go to the URL in your browser, you can see all the endpoints that are implemented. We will quickly go through the most important ones:

#### solutions/list 

This endpoint returns you a list of all the solutions that were found in the specified folder and that were able to be parsed. The list contains additional information about the solution that are provided in the `system.json` file of each solution.

In [ ]:
solutions = requests.get(f"{base_url}/solutions/list").json()
solutions

#### /solutions/{solution_name}/{scenario}/components

The next endpoint will provide you with all the components of a solution given a solution_name and a scenario. Each component contains details about its indices, if it is yearly, etc.

For example we can get the components of the first solution and the first scenario of the previous list:

In [ ]:
first_solution = solutions[0]
components = requests.get(f"{base_url}/solutions/{first_solution['name']}/{first_solution['scenarios'][0]}/components").json()
components

#### /solutions/get_data

Given the results of the previous requests, we can come to the core of the API, the `get_data` endpoint. It provides the values of the numpy-files after aggregating and filtering the data according to the request. The best way to see how the request has to be structured is by looking at the schema given under `http://localhost:8000/docs#/Solutions/get_data_solutions_get_data_post`. However, we will explain the functionalities here.

In general, the request must contain a solution name, a component, and a data_request. For now we leave the data_request empty which will result in the whole dataset. The response is parsed as a narrow csv, see https://en.wikipedia.org/wiki/Wide_and_narrow_data. This format is ideal for plotting libraries but further formats can be implemented without much effort.

In [ ]:
first_component = components[0]

request = {
    "solution_name": first_solution["name"],
    "component": first_component["component_name"],
    "data_request": {}
}

result = requests.post(f"{base_url}/solutions/get_data", json=request).json()
print(result)

In the example above, we did not further specify the data that we want. We can do this by adding index-sets to the data_request-field. For example if we only want the technologies "natural_gas_boiler" and "photovoltaics" we can add this to the request:

In [ ]:
request = {
    "solution_name": first_solution["name"],
    "component": first_component["component_name"],
    "data_request": {
        "index_sets": [
            {
                "index_title": "technology",
                "indices": ["natural_gas_boiler", "photovoltaics"]
            }
        ]
    }
}

result = requests.post(f"{base_url}/solutions/get_data", json=request).json()
print(result)

We can arbitrarily add more filters to it. Not specifying the indices simply selects all options:

In [ ]:
request = {
    "solution_name": first_solution["name"],
    "component": first_component["component_name"],
    "data_request": {
        "index_sets": [
            {
                "index_title": "technology"
            },
            {
                "index_title": "node",
                "indices": ["CH"]
            }
        ]
    }
}

result = requests.post(f"{base_url}/solutions/get_data", json=request).json()
print(result)

By default, each time series of the indices are returned separately. We can overwrite this behaviour by specifying the `behaviour` field in the script in order to sum over the indices. For example we can select the technologies `natural_gas_boiler` and `photovoltaics` and sum over the nodes `CH` and `DE`:

In [ ]:
request = {
    "solution_name": first_solution["name"],
    "component": first_component["component_name"],
    "data_request": {
        "index_sets": [
            {
                "index_title": "technology",
                "indices": ["natural_gas_boiler", "photovoltaics"]
            },
            {
                "index_title": "node",
                "indices": ["CH", "DE"],
                "behaviour": "sum"
            }
        ]
    }
}

result = requests.post(f"{base_url}/solutions/get_data", json=request).json()
print(result)

Since not specifying any indices at all selects all possible indices, we can for example sum over all time steps:

In [ ]:
request = {
    "solution_name": first_solution["name"],
    "component": first_component["component_name"],
    "data_request": {
        "index_sets": [
            {
                "index_title": "technology",
                "indices": ["natural_gas_boiler", "photovoltaics"]
            },
            {
                "index_title": "node",
                "behaviour": "sum",
                "indices": ["CH", "DE"]
            },
            {
                "index_title": "time_step",
                "behaviour": "sum"
            }
        ]
    }
}

result = requests.post(f"{base_url}/solutions/get_data", json=request).json()
print(result)

Currenly, NaN-Values are filtered and when a Sum contains NaN, the result will be NaN. This behaviour could easily be adapted. By default, the data are not aggregated. However, there are some components which are not yearly and there we can aggregate the values:

In [ ]:
request = {
    "solution_name": "PI",
    "component": "carbon_emissions_carrier",
    "data_request": {
        "index_sets": [
            {
                "index_title": "carrier",
                "indices": ["natural_gas"]
            },
            {
                "index_title": "node",
                "behaviour": "sum",
                "indices": ["CH", "DE"]
            },
        ]
    },
    "aggregate_years": True
}

result = requests.post(f"{base_url}/solutions/get_data", json=request).json()
print(result)